In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/INTER_IIT_2024_DATA")

Mounted at /content/drive


In [2]:
!ls

 BTC_2019_2023_12h.csv		     BTC_2019_2023_30m.csv  'ETHUSDT_1d (1).csv'
 BTC_2019_2023_15m.csv		     BTC_2019_2023_3d.csv    ETHUSDT_1h.csv
'BTC_2019_2023_1d (1).csv'	     BTC_2019_2023_3m.csv    ETHUSDT_2h.csv
 BTC_2019_2023_1d.csv		     BTC_2019_2023_4h.csv    ETHUSDT_30m.csv
 BTC_2019_2023_1d_with_signals.csv   BTC_2019_2023_6h.csv    ETHUSDT_3m.csv
'BTC_2019_2023_1h (1).csv'	     BTC_2019_2023_8h.csv    ETHUSDT_4h.csv
 BTC_2019_2023_1h.csv		     df_1m.csv		     ETHUSDT_5m.csv
 BTC_2019_2023_1m.csv		     df_download.csv	     ETHUSDT_6h.csv
 BTC_2019_2023_1month.csv	     downloaded_df.csv	     ETHUSDT_8h.csv
 BTC_2019_2023_1w.csv		     ETHUSDT_12h.csv	     result_df.csv
'BTC_2019_2023_2h (2).csv'	     ETHUSDT_15m.csv	     untrade-sdk


In [3]:
import numpy as np
import pandas as pd

# --------- DATA PREPROCESSING --------- #
def load_and_clean_data(file_path):

    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df[df['datetime'] >= '2020-01-01 00:00:00']                # Making sure the data is from 2020-01-01 00:00:00
                                                                    # Reset index and clean data (remove zero volume rows)
    df = df[df['volume'] != 0]
    df = df.reset_index(drop=True)

    return df

In [4]:
# --------- SIGNAL CREATION --------- #
def total_signal(df, current_candle):
   #Defining the pattern
    i = df.index.get_loc(current_candle)

    # Buy condition (pattern)
    buy_condition = all([
        df['high'].iloc[i] > df['high'].iloc[i-1],
        df['high'].iloc[i-1] > df['low'].iloc[i],
        df['low'].iloc[i] > df['high'].iloc[i-2],
        df['high'].iloc[i-2] > df['low'].iloc[i-1],
        df['low'].iloc[i-1] > df['high'].iloc[i-3],
        df['high'].iloc[i-3] > df['low'].iloc[i-2],
        df['low'].iloc[i-2] > df['low'].iloc[i-3]
    ])

    if buy_condition:
        return 1  # long signal

    # Sell condition (symmetrical pattern)
    sell_condition = all([
        df['low'].iloc[i] < df['low'].iloc[i-1],
        df['low'].iloc[i-1] < df['high'].iloc[i],
        df['high'].iloc[i] < df['low'].iloc[i-2],
        df['low'].iloc[i-2] < df['high'].iloc[i-1],
        df['high'].iloc[i-1] < df['low'].iloc[i-3],
        df['low'].iloc[i-3] < df['high'].iloc[i-2],
        df['high'].iloc[i-2] < df['high'].iloc[i-3]
    ])

    if sell_condition:
        return -1  # Short signal

    return 0  # No signal
                                                                #One Support/Resistance at a time(for the clear and holistic view of BTC PRICE)
def generate_signals(df):

    df['signals'] = df.apply(lambda row: total_signal(df, row.name), axis=1)         # NEW SIGNALS OTHER THAN PRIOR SIGNALS ARE BEING GENERATED HERE
    return df

# --------- SUPPORT/RESISTANCE AND SIGNAL MANAGEMENT --------- #
def stopSR(df):

    df['SR'] = np.zeros(len(df))

    for i in range(1, len(df)):
        if df['signals'].loc[i] == 1:
            df.loc[i, 'SR'] = df.loc[i, 'low']                     # Set SR for long positions (buy)
        elif df['signals'].loc[i] == -1:
            df.loc[i, 'SR'] = df.loc[i, 'high']                    # Set SR for short positions (sell)

    return df

def manage_signals_with_sl(df):

    df['signals_managed'] = df['signals'].copy()
    position = 0
    entry_price = 0

    i = 0
    while i < len(df):
        if df['signals'].iloc[i] == 1 and position == 0:
            position = 1
            entry_price = df['close'].iloc[i]

            for j in range(i + 1, len(df)):
                if df['low'].iloc[j] <= df['SR'].iloc[i]:         # SUPPORT triggered
                    df.at[j, 'signals_managed'] = -1
                    position = 0
                    i = j
                    break

        elif df['signals'].iloc[i] == -1 and position == 0:
            position = -1
            entry_price = df['close'].iloc[i]

            for j in range(i + 1, len(df)):
                if df['high'].iloc[j] >= df['SR'].iloc[i]:        # RESISTANCE triggered
                    df.at[j, 'signals_managed'] = 1
                    position = 0
                    i = j
                    break
        i += 1

    return df

def clean_up_signals(df):

    if 'signals' in df.columns:
        df = df.drop('signals', axis=1)  # Drop original signals

    if 'signals_managed' in df.columns:                 # Rename managed signals to 'signals' as they contain the new signals being generated after reaching the SUPPORT/RESISTANCE
        df = df.rename(columns={'signals_managed': 'signals'})

    if 'SR' in df.columns:
        df = df.drop('SR', axis=1)                     # Dropping SUPPORT/RESISTACNE as they arent of any use to us after signal generation

    return df


In [5]:
# --------- FILTER NOISE AND INCONSISTENCIES --------- #
def filter_trade_signals(df):

    cumulative_position = 0

    for i in range(len(df)):
        current_signal = df.loc[i, 'signals']

        if current_signal != 0:
            if cumulative_position == 0:
                cumulative_position = current_signal
            elif (cumulative_position > 0 and current_signal == -1) or (cumulative_position < 0 and current_signal == 1):
                cumulative_position = 0                          # Close position if signal opposes
            else:
                df.loc[i, 'signals'] = 0                         # Reset signal if position remains unchanged
        else:
            continue

    return df



In [6]:
# --------- MAIN SCRIPT --------- #
if __name__ == "__main__":
    # Load and clean data
    file_path = 'BTC_2019_2023_4h.csv'             # Replace with actual file path(USE 4H TIME FRAME FOR BTC)
    df = load_and_clean_data(file_path)

    # Generate initial signals
    df = generate_signals(df)

    # Apply support/resistance
    df = stopSR(df)

    # Manage signals with support/resistance logic
    df = manage_signals_with_sl(df)

    # Clean up the dataframe and final adjustments
    df = clean_up_signals(df)

    # Filter noisy signals
    df = filter_trade_signals(df)


In [7]:
df['datetime'] = pd.to_datetime(df['datetime'])                           #making sure that the datetime is in datetime format, to become compatible with UNTRADE

from google.colab import files

df.to_csv('result_df.csv', index=False)
files.download('result_df.csv')                                           #Downloading the final dataframe

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import os
import sys
!git clone https://github.com/ztuntrade/untrade-sdk.git
os.chdir('untrade-sdk')
!pip3 install .
untrade1= os.path.abspath('untrade-sdk')
sys.path.append(untrade1)
from untrade.client import Client
client = Client()

fatal: destination path 'untrade-sdk' already exists and is not an empty directory.
Processing /content/drive/MyDrive/INTER_IIT_2024_DATA/untrade-sdk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for untrade: filename=untrade-0.1.0-py3-none-any.whl size=5079 sha256=16566ba2cb84de9ead5673f93ab1907645436216ee30105c4887ae6208b033d5
  Stored in directory: /root/.cache/pip/wheels/2c/3c/fa/157295858821e9c5f1508f2f3473c1d1209bf7475ee9425b03
Successfully built untrade


In [9]:
import pandas as pd
import numpy as np
from pprint import pprint
import pandas as pd
import plotly.graph_objects as go
import datetime

In [10]:
# Function to plot trade signals on a candlestick chart
def plot_signals(csv_file):
    """
    Plots a candlestick or line chart with trade signals and reversals from a CSV file.

    Parameters:
    csv_file (str): Path to the CSV file containing the data.
    """
    # Load data from the CSV file
    data = pd.read_csv(csv_file, index_col='datetime', parse_dates=True)

    # Map trade types for better visualization and debugging
    data['trade_type'] = data['signals'].map({
        1: 'Long',
        -1: 'Short',
        0: 'Hold',
        2: 'Long Reversal',
        -2: 'Short Reversal'
    })

    # Create a figure for the chart
    fig = go.Figure()

    # Plot the price data as a candlestick chart
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        name='Candlestick',
        increasing_line_color='green',
        decreasing_line_color='red'
    ))

    # Plot Long (1) signals as green upward triangles
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == 1],
        y=data['close'][data['signals'] == 1],
        mode='markers',
        marker=dict(symbol='triangle-up', color='lime', size=10, line=dict(color='black', width=1)),
        name='Long (1)'
    ))

    # Plot Short (-1) signals as red downward triangles
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == -1],
        y=data['close'][data['signals'] == -1],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10, line=dict(color='black', width=1)),
        name='Short (-1)'
    ))

    # Plot Long Reversal (2) signals as cyan stars
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == 2],
        y=data['close'][data['signals'] == 2],
        mode='markers',
        marker=dict(symbol='star', color='cyan', size=10, line=dict(color='black', width=1)),
        name='Long Reversal (2)'
    ))

    # Plot Short Reversal (-2) signals as magenta stars
    fig.add_trace(go.Scatter(
        x=data.index[data['signals'] == -2],
        y=data['close'][data['signals'] == -2],
        mode='markers',
        marker=dict(symbol='star', color='magenta', size=10, line=dict(color='black', width=1)),
        name='Short Reversal (-2)'
    ))

    # Update layout to add titles and improve visualization
    fig.update_layout(
        title=f'BTC-USD Candlestick Chart with Trade Signals',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        xaxis_rangeslider_visible=False,
        legend=dict(y=1, traceorder='normal'),
        margin=dict(l=0, r=0, t=50, b=50),
    )

    # Display the chart
    fig.show()

# Function to calculate PnL and related data from a CSV file
def calculate_pnl_data(csv_file_path, initial_capital=1000):
    # Load data from the CSV file
    data = pd.read_csv(csv_file_path)

    # Initialize PnL-related columns
    data['entry'] = 0.0
    data['exit'] = 0.0
    data['quantity'] = 0.0
    data['pnl'] = 0.0
    data['Transaction_Cost'] = 0.0
    data['Net_Pnl'] = 0.0
    data['drawdown'] = 0.0
    trade_type = None
    data['trade_type'] = trade_type
    final_capital = initial_capital
    trades = 0

    # Loop through the data to find trades
    for i in range(len(data)):
        prev = data['signals'][:i].sum()
        entry = 0.0
        exit = 0.0
        quantity = 0.0
        trade_type = 'exit'

        if data.loc[i, 'signals'] != 0 or final_capital != 0:
            j = i + 1
            if data.loc[i, 'signals'] == 1 and prev == 0:  # Long entry
                while j < len(data) and (data.loc[j, 'signals'] not in [-1, -2]):
                    j += 1
                if j < len(data):
                    trades += 1
                    entry = data.loc[i, 'close']
                    exit = data.loc[j, 'close']
                    trade_type = 'long'
                    quantity = abs(final_capital / entry)
                    data.loc[i, 'quantity'] = quantity

            elif data.loc[i, 'signals'] == -1 and prev == 0:  # Short entry
                while j < len(data) and (data.loc[j, 'signals'] not in [1, 2]):
                    j += 1
                if j < len(data):
                    trades += 1
                    entry = data.loc[i, 'close']
                    exit = data.loc[j, 'close']
                    trade_type = 'short'
                    quantity = abs(final_capital / entry)
                    data.loc[i, 'quantity'] = quantity

            if j < len(data):  # Make sure we found an exit
                data.loc[i, 'entry'] = entry
                data.loc[i, 'exit'] = exit
                data.loc[i, 'quantity'] = quantity
                data.loc[i, 'trade_type'] = trade_type

                pnl = quantity * (exit - entry) if data.loc[i, 'signals'] > 0 else quantity * (entry - exit)
                transaction_cost = quantity * (entry) * 0.0015
                net_pnl = pnl - transaction_cost
                final_capital += net_pnl

                data.loc[i, 'pnl'] = pnl
                data.loc[i, 'Transaction_Cost'] = transaction_cost
                data.loc[i, 'Net_Pnl'] = net_pnl

    # Cumulative data
    data['Cumulative_PnL'] = data['pnl'].cumsum()
    data['Cumulative_Transaction_Cost'] = data['Transaction_Cost'].cumsum()
    data['Cumulative_Net_Pnl'] = data['Net_Pnl'].cumsum()

    # Calculate Buy-and-Hold and Equity curves
    buy_and_hold_start = data['close'].iloc[0]
    data['Buy_and_Hold'] = (data['close'] / buy_and_hold_start) * initial_capital
    data['Equity'] = initial_capital + data['Cumulative_Net_Pnl']

    # Calculate Drawdown
    data['Peak_Equity'] = data['Equity'].cummax()
    data['drawdown'] = (data['Equity'] - data['Peak_Equity']) / data['Peak_Equity']
    max_drawdown = data['drawdown'].min()

    # Save the updated DataFrame to a CSV file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"/content/Trade_Logs_{timestamp}.csv"
    data.to_csv(output_file, index=True)

    return data, output_file

# Final function to process the trading data
def backtest(csv_file_path, initial_capital=1000, start_date=None, end_date=None):
    # Plot signals
    plot_signals(csv_file_path)

    # Calculate PnL and filter data
    data, output_file = calculate_pnl_data(csv_file_path, initial_capital)

    if start_date:
        data = data[data['datetime'] >= start_date]
    if end_date:
        data = data[data['datetime'] <= end_date]

    print(f"Output File: {output_file}")

In [11]:
def perform_backtest(csv_file_path):

    # Create an instance of the untrade client
    client = Client()

    # Perform backtest using the provided CSV file path
    result = client.backtest(
        jupyter_id="socks",  # the one you use to login to jupyter.untrade.io
        file_path=csv_file_path,
        leverage=1,  # Adjust leverage as needed
    )

    return result

if __name__ == "__main__":

    # Prompt user to upload a file in Google Colab
    # print("Please upload your CSV file:")
    # uploaded = files.upload()

    # Get the uploaded file path
    # csv_file_path = list(uploaded.keys())[0]
    # print(f"File uploaded successfully: {csv_file_path}")

    csv_file_path = '/content/result_df - 2024-11-11T174317.085.csv'

    # Perform backtest using the uploaded CSV file
    backtest_result = perform_backtest(csv_file_path)

    # Display the backtest result
    print("Backtest results:")
    for value in backtest_result:
        print(value)

    backtest(csv_file_path)


Backtest results:
data: {
  "jupyter_id": "socks",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2020-01-01 00:00:00",
      "Total Trades": 41,
      "Leverage Applied": 1.0,
      "Winning Trades": 19,
      "Losing Trades": 22,
      "No. of Long Trades": 23,
      "No. of Short Trades": 18,
      "Benchmark Return(%)": 486.705254,
      "Benchmark Return(on $1000)": 4867.052543,
      "Win Rate": 46.341463,
      "Winning Streak": 3,
      "Losing Streak": 4,
      "Gross Profit": 5323.786115,
      "Net Profit": 5262.286115,
      "Average Profit": 128.348442,
      "Maximum Drawdown(%)": 4.830413,
      "Average Drawdown(%)": 1.061331,
      "Largest Win": 3786.062494,
      "Average Win": 326.177495,
      "Largest Loss": -164.21532,
      "Average Loss": -42.503922,
      "Maximum Holding Time": "245 days 12:0:0",
      "Average Holding Time": "26 days 19:42:26",
      "Maximum Adverse Excursion": 23.469801,


Output File: /content/Trade_Logs_20241111_121413.csv
